In [1]:
from pycparser import parse_file, c_ast
from numpy import inf
from copy import copy

In [2]:
def parse_c_file(file):
    """
        Parse a c_file via pycparser
        
        Arguments
        ----------
        file: String
            c file to parse
    """
    ast = parse_file(file, use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return ast

In [3]:
ast_graph = parse_c_file('./C files/if.c')
foo_graph = ast_graph.ext[0]

RuntimeError: Unable to invoke 'cpp'.  Make sure its path was passed correctly
Original error: [WinError 2] Le fichier spécifié est introuvable

In [ ]:
class Value():
    """
        Représente une valeur de variable qui est sous la forme: String + Constante
        
        Attributs
        ----------
        constante: Int
            Constante
        variable: Str
            name variable
    """
    def __init__(self, constante = 0, variable = None):
        self.variable = variable
        self.constante = constante
    
    def compute(ast_node):
        if type(ast_node) is c_ast.Constant:
            return Value(constante = int(ast_node.value))
        
        elif type(ast_node) is c_ast.ID:
            return Value(variable=ast_node.name)
        
        elif type(ast_node) is c_ast.BinaryOp:
            left, right, op = ast_node.left, ast_node.right, ast_node.op 
            
            if type(left) is c_ast.ID and type(right) is c_ast.ID:
                raise Exception("Operation of two variables")
            elif type(left) is c_ast.Constant and type(right) is c_ast.ID:
                return Value(variable=right.name, constante=int(left.value))
            elif type(left) is c_ast.ID and type(right) is c_ast.Constant:
                return Value(variable=left.name, constante=int(right.value))
            else:
                print(ast_node)
                if op is "+":
                    return Value(constante=(int(left.value)+int(right.value)))
                elif op is "-":
                    return Value(constante=(int(left.value)-int(right.value)))          
        
    def __str__(self):
        if self.variable is None:
            return str(self.constante)
        return self.variable + ' + ' + str(self.constante)

In [ ]:
print(Value(-10, 'a'))

In [ ]:
class Variable():
    """
        A variable is an interval or a singleton
    """
    def __init__(self):
        pass

class Singleton(Variable):
    """
        Représente un singleton
        
        Attributs
        ----------
        value: @Value
            valeur du singleton
    """
    def __init__(self, value):
        super()
        self.value = value
    
    def __str__(self):
        return ("{" + str(self.value) + "}")
    
class Interval(Variable):
    """
        Représente un interval
        
        Attributs
        ----------
        min: @Value
            Minimum de l'intervalle
        max: @Value 
            Maximum de l'intervalle
    """
    def __init__(self, minimum, maximum):
        self.min = minimum
        self.max = maximum
    
    def __str__(self):
        return ("[" + str(self.min) + "," + str(self.max) + "]")

In [ ]:
a = Interval(Value(constante=-inf), Value(constante=inf))
b = Singleton(Value(constante=0))

print(a, b)

In [ ]:
class Condition:
    """
        Represents a condition
        
        Arguments
        ----------
        coord: String
            lines of the condition
        left: @Variable
            left operand
        operator: String
            operator
        right: @Variable
            right operand
    """
    def __init__(self, condition = None, condition_boolean = True):
        self.condition = condition
        (self.left, self.operator, self.right) = self.handleCondition()
    
    def handleCondition(self):
        left_operand = Singleton(Value.compute(self.condition.left))
        right_operand = Singleton(Value.compute(self.condition.right))
        operator = self.condition.op
        
        if operator == ">":
            return (right_operand, "<", left_operand)
        elif operator == ">=":
            return (right_operand, "<=", left_operand)
        elif operator == "%":
            raise Exception("Modulo is not defined")
        else:
            return (left_operand, operator, right_operand)
    
    def reverse(condition):
        reversed_condition = copy(condition)
        reversed_condition.condition_boolean = !condition.condition_boolean
        if condition.operator == "<":
            reversed_condition.left, reversed_condition.operator, reversed_condition.right = \
                condition.right, "<=", condition.left
        elif condition.operator == "<=":
            reversed_condition.left, reversed_condition.operator, reversed_condition.right = \
                condition.right, "<", condition.left
        elif condition.operator == "==":
            reversed_condition.operator = "!="
        elif condition.operator == "!=":
            reversed_condition.operator = "=="
        else:
            raise Exception("Wrong operator for conditions: " + condition.operator)
        return reversed_condition
    
    def __str__(self):
        return str(self.left) + " " + self.operator + " " + str(self.right)

In [ ]:
if_condition = foo_graph.body.block_items[3].cond
condition = Condition(if_condition)
condition2 = Condition.reverse(condition)
print(condition)
print(condition2)

In [ ]:
class Context:
    """
        Context of execution of the program
        
        Arguments
        ----------
        conditions: list[@Condition]
            conditions evaluated by the context
        variables: list[@Variable]
            variables in the context
        next_nodes: list[@c_ast.Node]
            next nodes to visit
        state: boolean
            true if the context executes, false if it fails
    """
    def __init__(self, condition = None, next_nodes = [], previous_context = None):
        if previous_context is not None:
            self.conditions = copy(previous_context.conditions) + [condition]
            self.variables = copy(previous_context.variables)
            self.next_nodes = copy(previous_context.next_nodes) + next_nodes
            self.checkCondition(condition)
        else:
            self.conditions = []
            self.variables = {}
            self.next_nodes = next_nodes
    
    def addParam(self, param):
        name = param.name
        types = param.type.type.names
        if 'int' not in types:
            raise Exception('Only int type allowed')
        if 'unsigned' in types:
            self.variables[name] = Interval(Value(constante = 0), Value(constante = inf))
        else:
            self.variables[name] = Interval(Value(constante = -inf), Value(constante = inf))
    
    def addVariable(self, node_decl):
        name = node_decl.name
        types = node_decl.type.type.names
        if 'int' not in types:
            raise Exception('Only int type allowed')
        if node_decl.init is not None:
            variable = Singleton(Value.compute(node_decl.init))
        else:
            variable = Singleton(Value(constante=0))
        self.variables[name] = variable
    
    def assign(self, node_assign):
        name = node_assign.lvalue.name
        variable = Singleton(Value.compute(node_assign.rvalue))
        self.variables[name] = variable
    
    def pop(self):
        if len(self.next_nodes) > 0:
            pop_res = self.next_nodes.pop()
            if pop_res is None:
                return self.pop()
            return pop_res
        return None
    
    def computeVariable(self, variable):
        if type(variable) is Singleton:
            if variable.value.variable is not None:
                child_variable = self.variables[variable.value.variable]
                if type(child_variable) is Singleton:
                    if child_variable.value.variable is not None:
                        child2 = self.variables[child_variable.value.variable]
                        if type(child2) is Singleton:
                            if child2.value.variable is not None:
                                raise Exception("Une valeur dépend de plus de deux variables")
                            else:
                                return Singleton(Value(constante = variable.value.constante +\
                                                                   child_variable.value.constante\
                                                                   + child2.value.constante))
                        elif type(child2) is Interval:
                            if child2.min.variable is not None or child2.min.variable is not None:
                                raise Exception("Une valeur dépend de plus de deux variables")
                            else:
                                return Interval(Value(constante = variable.value.constante +\
                                                                  child_variable.value.constante\
                                                                  + child2.min.constante),\
                                                Value(constante = variable.value.constante +\
                                                                  child_variable.value.constante\
                                                                  + child2.max.constante))       
                    else:
                        return Singleton(Value(constante = variable.value.constante +\
                                                     child_variable.value.constante))
                elif type(child_variable) is Interval:
                    min_child, max_child = child_variable.min, child_variable.max
                    if min_child.variable is not None or max_child.variable is not None:
                        raise Exception("Une valeur dépend de plus de deux variables")
                    else:
                        return Interval(Value(constante = variable.value.constante +\
                                                          min_child.constante),
                                        Value(constante = variable.value.constante +\
                                                          max_child.constante))
            else:
                return variable
        else:
            raise Exception("Compute variable should take as parameter a Singleton")

    def checkCondition(self, condition):
        """
        Si self.state = True condition possible
        Si self.state = False condition impossible donc le branchement est inutile
        """
        self.state = True
        # Si condition sous la forme: constante op constante
        if condition.left.value.variable is None and condition.right.value.variable is None:
            self.state = Operation.compareConstante(condition.left.value.constante,\
                                                    condition.right.value.constante,\
                                                    condition.operator)
        # Si condition sous la forme: variable + constante op variable + constante
        elif condition.left.value.variable is not None and\
             condition.right.value.variable is not None:
            # Si condition sous la forme: a + constante op a + constante
            if condition.left.value.variable == condition.right.value.variable:
                 self.state = Operation.compareConstante(condition.left.value.constante,\
                                                    condition.right.value.constante,\
                                                    condition.operator)
            # Si condition sous la forme: a + constante op b + constante
            else:
                # Si condition sous la forme:
                    # {b + constante} + constante op b + constante
                if type(self.variables[condition.left.value.variable]) is Singleton and\
                   self.variables[condition.left.value.variable].value.variable == condition.right.value.variable\
                   or type(condition.left.value.variable) is Interval:
                    self.state = Operation.compareConstante(condition.left.value.constante +\
                                                        self.variables[condition.left.value.variable].value.constante,\
                                                        condition.right.value.constante,\
                                                        condition.operator)
                    # a + constante op {a + constante} + constante
                elif type(self.variables[condition.right.value.variable]) is Singleton and\
                     self.variables[condition.right.value.variable].value.variable == condition.left.value.variable\
                     or type(condition.left.value.variable) is Interval:
                        self.state = Operation.compareConstante(condition.left.value.constante,
                                                        condition.right.value.constante +\
                                                        self.variables[condition.right.value.variable].value.constante,\
                                                        condition.operator)
                pass
        return
    
    
    def checkCondition(self, condition):
        value = condition.left.value
        const1 = value.constante
        listVar1 = []
        while (value.variable is not None):
            listVar1.append(value.variable)
            value = self.variables[value.variable].value
            const1 += value.constante
            
        value = condition.right.value
        const2 = 0
        interval2 = None
        condition.addVariable(listVar1, value, const2, interval2)
        
        interval1 = None
        for var in listVar1:
            addVariable(None, var.value, const1, interval1)
            
        if (interval1 is None and interval2 is None):
            self.state = Operation.compareConstante(const1, const2, condition.operator)
        elif (interval1 is not None and interval2 is not None):
            if (condition.operator == "<"):
                if (interval1.min + const1 >= interval2.max + const2):
                    return False
                elif (interval2.max + const1 < interval2.min + const2):
                    return True
                else:
                    pass
            if (condition.operator == "<="):
                if (interval1.min + const1 > interval2.max + const2):
                    return False
                elif (interval2.max + const1 <= interval2.min + const2):
                    return True
                else:
                    pass
                
    def addVariable(self, listVar1, value, const, interval):
        const += value.constante
        if (value.variable is in listVar1):
                listVar1.remove(value.variable)
        else:
            variable = self.variables[value.variable]
            if (type(variable) is Singleton):
                addVariable(listVar1, value.variable.value, const, interval)
            else if (type(variable) is Interval):
                if (interval is None):
                    interval = Interval(0, 0)
                if (variable.min == -inf):
                    interval.min = -inf
                if (interval.min != -inf):
                    interval.min = getMin(variable.min)

                if (variable.max == inf):
                    interval.max = inf
                if (interval.max != inf):
                    interval.max = getMax(variable.max)
                        
    def getMin(self, value):
        minimum = value.constante
        if (value.variable is None):
            return minimum
        variable = self.variables[value.variable]
        if (type(variable) is Singleton):
            return minimum = minimum + getMin(variable.value)
        elif (type(variable) is Interval):
            if (variable.min == -inf):
                return -inf
            else:
                return minimum + getMin(variable.min)
            
    def getMax(self, value):
        maximum = value.constante
        if (value.variable is None):
            return maximum
        variable = self.variables[value.variable]
        if (type(variable) is Singleton):
            return maximum = maximum + getMax(variable.value)
        elif (type(variable) is Interval):
            if (variable.max == inf):
                return inf
            else:
                return maximum + getMax(variable.max)
    
    def __str__(self):
        return "Context: [conditions: " + \
               ", ".join(str(condition) for condition in self.conditions) + \
               "]; [variables: " + ", ".join(variable + ":" + str(self.variables[variable]) \
                                          for variable in self.variables) + "]"

In [ ]:
class Operation():
    
    def compareConstante(left, right, operator):
        if operator == "<" and left >= right:
            return False
        elif operator == "<=" and left > right:
            return False
        elif operator == "==" and left != right:
            return False
        elif operator == "!=" and left == right:
            return False
        return True
    
    def computeVariable(variable):
        if type(variable) is Singleton:
            if variable.variable is None:
                return variable
            else:
                if type(variable.variable) is Singleton:
                    return 

In [ ]:
def isIf(node):
    return type(node) == c_ast.If

def isDeclaration(node):
    return type(node) == c_ast.Decl

def isAssignment(node):
    return type(node) == c_ast.Assignment

def getCoord(node):
    return node.coord

In [ ]:
def visitParamsFunction(context, params):
    for param in params:
        context.addParam(param)

In [ ]:
def visitNode(context):
    node = context.pop()
    if node is not None:
        if isIf(node):
            condition = Condition(node.cond)
            context1 = Context(condition = condition, next_nodes = [node.iftrue],\
                               previous_context = context)
            context2 = Context(condition = Condition.reverse(condition),\
                               next_nodes = [node.iffalse], previous_context = context)
            if context1.state and context2.state:
                return visitNode(context1) + visitNode(context2)
            elif context1.state and not context2.state:
                return visitNode(context1) + [context2]
            elif not context1.state and context2.state:
                return visitNode(context2) + [context1]
            else:
                return [context1, context2]
        elif isDeclaration(node):
            context.addVariable(node)
        elif isAssignment(node):
            context.assign(node)
        else:
            children = copy(node.children())
            for (string, child) in reversed(children):
                context.next_nodes.append(child)
        return visitNode(context)
    print(context)
    return []

In [ ]:
def visitFunction(function):
    context = Context(next_nodes = [function.body])
    visitParamsFunction(context, function.decl.type.args.params)
    return visitNode(context)

In [ ]:
contexts = visitFunction(foo_graph)

print("\n")
for context in contexts:
    print("La condition: " + str(context.conditions[-1]) + " n'est pas possible à la ligne " +\
          str(context.conditions[-1].condition.coord))
    print(context)
    print("\n")